In [1]:
import os
import pandas as pd
from autograd import grad
import autograd.numpy as np
import numpy as np1
import numpy.matlib
from numpy.linalg import inv
from numpy import linalg as LA
from numpy.matlib import repmat

In [2]:
import sys
sys.path.append('../../Python/')
from Preprocessing import standardize
from Evaluation import split_train_test_ma, train_model, test_model
from Databases import get_databases_path, get_database

In [3]:
databases_path = '../../Databases/Sinteticas'
paths = get_databases_path(databases_path)

In [4]:
# Obtain the attributes and labels
attr_df = get_database(paths[1], 'database_attr.csv')
labels_df = get_database(paths[1], 'database_labels.csv')

In [5]:
# Preprocessing the attributes
standardized_attr_df = standardize(attr_df)

In [6]:
# Split in Training and Test Sets
X_train, X_test, y_train, y_test = split_train_test_ma(standardized_attr_df, labels_df)

In [7]:
def sigmoid(a):
    return 1./(1 + np.exp(-a))

def make_cost_function(X, Y, mui):
    def cost_function(w):
        R = Y.shape[1]
        N = Y.shape[0]
        sizew1 = X.shape[1]
        sizeW = sizew1 + sizew1*R

        W1 = w[0,0:sizew1].reshape(sizew1,1)
        W2 = w[0,sizew1:sizeW].reshape(sizew1,R)

        pi = sigmoid(np.dot(X, W1))        
        aux = sigmoid(np.dot(X,W2))
        
        ai = np.multiply(np.power((1-aux),np.absolute(Y-1)),np.power(aux, (1-np.absolute(Y-1))))
        ai = np.array(np.prod(ai, axis=1)).reshape(ai.shape[0],1) 

        bi = np.multiply(np.power((1-aux),np.absolute(Y)),np.power(aux, (1-np.absolute(Y))))
        bi = np.array(np.prod(bi, axis=1)).reshape(bi.shape[0],1) 

        aux_0 = 1-mui 
        aux_1 = 1-pi 

        aux_a = np.multiply(mui, np.log(np.multiply(pi, ai))) 
        aux_b = np.multiply(aux_0, np.log(np.multiply(aux_1, bi))) 

        return -(aux_a + aux_b).sum()
    return cost_function
         
def optimizer(X, Y, w_0, mui):
    derivate = grad(make_cost_function(X, Y, mui)) 
    params = w_0
    epsilon = 0.001
    normGrad = 100
    i=0
    while normGrad > 0.1 and i<300:
        gr = derivate(params)
        params = params - epsilon * gr
        normGrad = LA.norm(gr)
        i+=1
    return params, normGrad
    
def train_mae(X, Y):
    X.loc[:,'b'] = np.zeros((X.shape[0],1))
    X = X.values
    Y = Y.values
    R = Y.shape[1] # Number of annotators.
    D = X.shape[1] # Input space dimension
    N = X.shape[0] # Number of samples.
    
    sizew1 = D
    sizew2 = D*R;
    sizeW = sizew1 + sizew2
    
    w = np.random.randn(1, sizeW)
    mui = np.array(Y.mean(axis=1)).reshape(Y.shape[0],1)
    
    w = repmat(np.dot(inv(np.dot(X.T,X) + 1e-20*np.eye(D)), np.dot(X.T,mui)),1,R+1)
    w = w.flatten().reshape(1, sizeW)
    
    w_0 = w
    loglik1 = 10000000
    difloglik = 100
    difloglik1 = -1
    i = 0
    
    while abs(difloglik) > 0.2 and i<100:
        # M-step     
        w, Ngr = optimizer(X, Y, w_0, mui)
        W1 = w[0,0:sizew1].reshape(sizew1,1)
        W2 = w[0,sizew1:sizeW].reshape(D,R)

        #E-step
        pi = sigmoid(np.dot(X, W1))
        aux = sigmoid(np.dot(X,W2))

        ai = np.multiply(np.power((1-aux),np.absolute(Y-1)),np.power(aux, (1-np.absolute(Y-1))))
        ai = np.array(np.prod(ai, axis=1)).reshape(ai.shape[0],1) 

        bi = np.multiply(np.power((1-aux),np.absolute(Y)),np.power(aux, (1-np.absolute(Y))))
        bi = np.array(np.prod(bi, axis=1)).reshape(bi.shape[0],1) 

        nummui = np.multiply(ai, pi)
        denmui = (np.multiply(ai, pi) + np.multiply(bi, (1-pi))) 
        mui =  np.divide(nummui, denmui)

        aux_0 = 1-mui 
        aux_1 = 1-pi 
        aux_a = np.multiply(mui, np.log(np.multiply(pi, ai))) 
        aux_b = np.multiply(aux_0, np.log(np.multiply(aux_1, bi))) 

        loglik2 = -(aux_a + aux_b).sum()
        difloglik = (loglik2 - loglik1);
        loglik1 = loglik2
        print('Verosimilitud:', abs(difloglik))
        
        w_0 = w
        i+=1
    return w[0,0:sizew1].reshape(sizew1,1) 

In [8]:
W1 = train_mae(X_train, y_train)

Verosimilitud: nan


/Users/codigoriginal/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/codigoriginal/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:635: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/Users/codigoriginal/anaconda3/lib/python3.7/site-packages/autograd/tracer.py:48: RuntimeWarning: divide by zero encountered in log
  return f_raw(*args, **kwargs)
/Users/codigoriginal/an

In [9]:
X_test.loc[:,'b'] = 0.0
pred = sigmoid(np.dot(X_test, W1))
y_pred = [True if item >= 0.5 else False for item in pred]

In [10]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print("Global Precision:")
print(accuracy_score(y_test, y_pred))
print("\n")

print("General Report:")
print(classification_report(y_test, y_pred, target_names=['yes','no']))
print("\n")

print("Confusion Matrix: ")
matriz_confusion = confusion_matrix(y_test, y_pred)
table = pd.DataFrame(matriz_confusion)
print(table)

Global Precision:
0.3225806451612903


General Report:
              precision    recall  f1-score   support

         yes       0.32      1.00      0.49        30
          no       0.00      0.00      0.00        63

    accuracy                           0.32        93
   macro avg       0.16      0.50      0.24        93
weighted avg       0.10      0.32      0.16        93



Confusion Matrix: 
    0  1
0  30  0
1  63  0


/Users/codigoriginal/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
